In [ ]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import sys
from matplotlib import animation, rc
from IPython.display import HTML, Image
import pandas as pd

src_dir = os.path.abspath("/home/mmfarrugia/repos/optimization/hybrid_optimizer")
sys.path.append(src_dir)

from hybrid_optimizer import PSO_DE
from pyswarms.utils.plotters.formatters import Mesher, Designer
from pyswarms.utils.plotters.plotters import plot_cost_history, plot_contour
from plotters import plot_summary, plot_surface
import example_funcs

technical_replicates = 10
size_pop = 50
max_iter = 200

In [ ]:
# Universal config setup
random_config = {
    'name': 'Random',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}
random_fixedF_config = {
    'name': 'RandomFixedF',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "F": (0.5, 0.5),
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}
best_config = {
    'name': 'Best',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}
best_fixedF_config = {
    'name': 'BestFixedF',
    "n_dim": 2,
    "F": (0.5, 0.5),
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}

taper_rand_config = {
    'name': 'RandTaperFreq',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": True,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}

taper_best_config = {
    'name': 'BestTaperFreq',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": True,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}

configs = [random_config, random_fixedF_config, best_config, best_fixedF_config, taper_rand_config, taper_best_config]

In [ ]:
def run_plot_opts(function, constraint_uneq, dimensions, config_list, technical_replicates, name):
    fig, ax = plt.subplots(1, len(config_list), figsize=(5.5*len(config_list),6), sharey=True)
    fig.suptitle(name)
    opts = {}
    full_opts = {}
    for i, config in enumerate(config_list):
        opt_name = config['name']
        full_opts[opt_name] = []
        opt = (PSO_DE(function, n_dim=dimensions, config=config))
        opt.record_mode = True
        opt.run()
        print('i: best_x is ', opt.gbest_x, 'best_y is', opt.gbest_y)
        opts[opt_name] = opt
        full_opts[opt_name].append(opt)
        sum_loss = np.array(opt.gbest_y_hist)
        sum_x = np.array(opt.gbest_x)
        sum_y = opt.gbest_y
        for t in range(technical_replicates-1):
            opt = (PSO_DE(function, n_dim=dimensions, config=config, constraint_ueq=constraint_uneq))
            opt.record_mode = True
            opt.run()
            full_opts[opt_name].append(opt)
            sum_loss += opt.gbest_y_hist
            sum_x += opt.gbest_x
            sum_y += opt.gbest_y
        #fig.add_subplot(1, len(configs), i+1)
        average_loss = np.divide(sum_loss, technical_replicates)
        average_x = np.divide(sum_x, technical_replicates)
        average_y = sum_y / technical_replicates
        #average_x = ["{:.5e}".format(x) for x in average_x]
        results_text =  "{:.5e}".format(average_y[0]) + ' @ X: ' + str(average_x)
        ax[i].plot(average_loss)
        #plt.xlabel('Iteration')
        ax[i].set_ylabel('Best Known F(X)')
        ax[i].annotate(results_text, (0,0), (0,-20), xycoords='axes fraction', textcoords='offset points', va='top')
        ax[i].set_title(opt_name)

    fig.tight_layout()
    plt.show()

    return opts, full_opts

In [ ]:
def plot_swarm_w_loss(opts, config_list, title="Swarm Diversity & Loss"):
    Y_history = []
    fig, ax = plt.subplots(2, len(opts), figsize=(5*len(config_list),8), sharey='row')
    fig.suptitle(title)
    for i, key in enumerate(opts):
        opt = opts[key]
        Y_history = pd.DataFrame(np.array(opt.record_value['Y']).reshape((-1, opt.size_pop)))
        ax[0,i].plot(Y_history.index, Y_history.values, '.')
        ax[0,i].set_title(config_list[i]['name'])
        Y_history.min(axis=1).cummin().plot(kind='line', ax=ax[1,i])
        print(len(Y_history))
    plt.show()

Himmelblau's  objective function

    Only takes two dimensions and has a four equal global minimums
     of zero at :code:`f([3.0,2.0])`, :code:`f([-2.805118,3.131312])`,
     :code:`f([-3.779310,-3.283186])`, and :code:`f([3.584428,-1.848126])`.
    Its coordinates are bounded within :code:`[-5,5]`.

    Best visualized with the full domain and a range of :code:`[0,1000]`

In [ ]:
n_dim = 2
lb = [-5.0, -5.0]
ub = [5.0, 5.0]
size_pop = 100
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))
global_opt = (-3.779310,-3.283186, 0.)

In [ ]:
from plotters import plot_cost_history, plot_contour, plot_surface, Mesher, Designer
#Plot the sphere function's mesh for better plots
m = Mesher(func=example_funcs.himmelblau, limits=[(-5,5), (-5,5)], levels = [1e-5, 1e-4, 0.001, 0.01, 0.1, 1., 10, 100, 1000])
#Adjust figure limits
d = Designer(limits=[(-5,5), (-5,5), (0,1000)], label=['x-axis', 'y-axis', 'z-axis'])

In [ ]:
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))

for config in configs:
    config["initial_guesses"] = common_start
    config["lb"] = lb
    config["ub"] = ub
    config["n_dim"] = n_dim
    config["vectorize_func"] = False


In [ ]:
opts, full_opts = run_plot_opts(example_funcs.himmelblau, [lambda x: x[1]-x[0]-1, lambda x: x[0]-1-x[1]], n_dim, configs, technical_replicates, 'Himmelblau Constrained\nAverage Loss Over Optimization with '+ str(technical_replicates) + ' Technical Replicates')

In [ ]:
plot_swarm_w_loss(opts, configs, 'Himmelblau Constrained')

'Rand' Mutation
/DE/rand/1

In [ ]:
ho_rand_1 = PSO_DE(example_funcs.himmelblau, constraint_ueq=[lambda x: x[1]-x[0]-1, lambda x: x[0]-1-x[1]],  n_dim=2, config=random_config)
ho_rand_1.record_mode = True
ho_rand_1.run()
print('best_x is ', ho_rand_1.gbest_x, 'best_y is', ho_rand_1.gbest_y)
plt.plot(ho_rand_1.gbest_y_hist)
plt.show()

'Best' Mutation
/DE/best/1

In [ ]:
ho_best_1 = PSO_DE(example_funcs.himmelblau, constraint_ueq=[lambda x: x[1]-x[0]-1, lambda x: x[0]-1-x[1]], n_dim=2, config=best_config)
ho_best_1.record_mode = True
ho_best_1.run()
print('best_x is ', ho_best_1.gbest_x, 'best_y is', ho_best_1.gbest_y)
plt.plot(ho_best_1.gbest_y_hist)
plt.show()

In [ ]:
import pandas as pd

Y_history_rand = pd.DataFrame(np.array(ho_rand_1.record_value['Y']).reshape((-1, ho_rand_1.size_pop)))
Y_history_best = pd.DataFrame(np.array(ho_best_1.record_value['Y']).reshape((-1, ho_best_1.size_pop)))
fig, ax = plt.subplots(2, 2)
fig.suptitle('Himmelblau')
ax[0,0].set_title('DE/rand/1')
ax[0,1].set_title('DE/best/1')
ax[1,0].set_title(str(ho_rand_1.gbest_y)+' @ X: '+str(ho_rand_1.gbest_x))
ax[1,1].set_title(str(ho_best_1.gbest_y)+' @ X: '+str(ho_best_1.gbest_x))
ax[0,0].plot(Y_history_rand.index, Y_history_rand.values, '.', color='red')
ax[0,1].plot(Y_history_best.index, Y_history_best.values, '.', color='red')
Y_history_rand.min(axis=1).cummin().plot(kind='line', ax=ax[1,0])
Y_history_best.min(axis=1).cummin().plot(kind='line', ax=ax[1,1])
plt.show()

In [ ]:
%%capture plsGod

Y_history_rand = pd.DataFrame(np.array(ho_rand_1.record_value['Y']).reshape((-1, ho_rand_1.size_pop)))
Y_history_best = pd.DataFrame(np.array(ho_best_1.record_value['Y']).reshape((-1, ho_best_1.size_pop)))
fig, ax = plt.subplots(3, 2)
fig.suptitle('Himmelblau')
ax[0,0].set_title('DE/rand/1')
ax[0,1].set_title('DE/best/1')
ax[1,0].set_title(str(ho_rand_1.gbest_y)+' @ X: '+str(ho_rand_1.gbest_x), fontsize=8)
ax[1,1].set_title(str(ho_best_1.gbest_y)+' @ X: '+str(ho_best_1.gbest_x))
ax[0,0].plot(Y_history_rand.index, Y_history_rand.values, '.', color='red')
ax[0,1].plot(Y_history_best.index, Y_history_best.values, '.', color='red')
Y_history_rand.min(axis=1).cummin().plot(kind='line', ax=ax[1,0])
Y_history_best.min(axis=1).cummin().plot(kind='line', ax=ax[1,1])
plot_contour(pos_history=ho_rand_1.record_value['X'], mesher=m, designer=d, canvas=(fig, ax[2,0]))
#plot_contour(pos_history=ho_best_1.record_value['X'], mesher=m, designer=d, ax=ax[2,1])
plt.show()


In [ ]:
%%capture

#Make animation
animation2D_rand = plot_contour(pos_history=ho_rand_1.record_value['X'], mesher=m, designer=d)


In [ ]:
animation2D_rand.save('himmelblau_constrained_rand.gif', writer='ffmpeg', fps=15)

In [ ]:
Image('himmelblau_constrained_rand.gif')

In [ ]:
%%capture

#Make animation
animation2D_best = plot_contour(pos_history=ho_best_1.record_value['X'], mesher=m, designer=d)


In [ ]:
animation2D_best.save('himmelblau__constrained_best.gif', writer='ffmpeg', fps=15)

In [ ]:
Image('himmelblau_constrained_best.gif')

In [ ]:
from PIL import Image as img
fig, ax = plt.subplots(1, 2, figsize=(10,7))
fig.suptitle('Himmelblau', fontsize=24)
ax[0].set_title('DE/rand/1')
ax[1].set_title('DE/best/1')
  
# showing image 
ax[0].imshow(np.asarray(img.open('himmelblau_constrained_rand.gif'))) 
ax[0].axis('off') 

ax[1].imshow(np.asarray(img.open('himmelblau__constrained_best.gif'))) 
ax[1].axis('off')

fig.tight_layout()
# fig.add_subplot(rows, columns, 2) 
# plt.imshow(Image('himmelblau_best.gif')) 
# plt.axis('off') 
# plt.title("DE/best/1")

In [ ]:
pos_history_3d_rand = m.compute_history_3d(ho_rand_1.record_value['X']) #preprocessing

In [ ]:
%%capture

animation3d_rand = plot_surface(pos_history=pos_history_3d_rand, mesher=m, designer=d)
plt.show()


In [ ]:
animation3d_rand.save('himmelblau_constrained_3d_rand.gif', writer='ffmpeg', fps=15)
Image(url='himmelblau_constrained_3d_rand.gif')

In [ ]:
pos_history_3d_best = m.compute_history_3d(ho_best_1.record_value['X']) #preprocessing

In [ ]:
%%capture

animation3d_best = plot_surface(pos_history=pos_history_3d_best, mesher=m, designer=d)
plt.show()


In [ ]:
animation3d_best.save('himmelblau_constrained_3d_best.gif', writer='ffmpeg', fps=15)
Image(url='himmelblau_constrained_3d_best.gif')

In [ ]:
opts = [ho_rand_1, ho_best_1]

In [ ]:
%%capture

ani_summary = plot_summary(optimizers=opts, title="Himmelblau", titles=[opt.mutation_strategy for opt in opts],  n_processes=8, mesher=m, designer=d)
plt.show()

In [ ]:
ani_summary.save('himmelblau_constrained_fig.gif', writer='ffmpeg', fps=15)
Image(url='himmelblau_constrained_fig.gif')